In [1]:
import numpy as np
import pandas as pd
#import sqlite3 as sq
import re
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

In [2]:
def label_cleaner(text):
        
    text = re.sub( r'ЖАЛОБА: на ', '', text) # Убрать из темы словосочетания 'ЖАЛОБА: на ',
    text = re.sub( r'ЖАЛОБА: ', '', text) # 'ЖАЛОБА: ',
    text = re.sub( r'Жалоба на ', '', text) # 'Жалоба на ',
    text = re.sub( r'Жалобы на ', '', text) # 'Жалобы на '
    
    text = str.strip(text.lower()) #Переводим в нижний регистр и убираем пробелы
    return  text

def text_cleaner(text):
    text = text.lower() # приведение в lowercase,
    
    text = re.sub( r'https?://[\S]+', ' url ', text) # замена интернет ссылок
    text = re.sub( r'[\w\./]+\.[a-z]+', ' url ', text) 
 
    # text = re.sub( r'\d+[-/\.]\d+[-/\.]\d+', ' date ', text) # замена даты и времени
    # text = re.sub( r'\d+ ?гг?', ' date ', text) 
    # text = re.sub( r'\d+:\d+(:\d+)?', ' time ', text) 

    # text = re.sub( r'@\w+', ' tname ', text ) # замена имён twiter
    # text = re.sub( r'#\w+', ' htag ', text ) # замена хештегов

    text = re.sub( r'<[^>]*>', ' ', text) # удаление html тагов
    text = re.sub( r'[\W]+', ' ', text ) # удаление не словарных символов

    stemmer = SnowballStemmer("russian")
    singles = [stemmer.stem(word) for word in text.split()]
    text = ' '.join(singles)

    # stw = ['в', 'по', 'на', 'из', 'и', 'или', 'не', 'но', 'за', 'над', 'под', 'то',
    #        'a', 'at', 'on', 'of', 'and', 'or', 'in', 'for', 'at' ]
    # remove = r'\b('+'|'.join(stw)+')\b'
    # text = re.sub(remove,' ', text)
    
    # text = re.sub( r'\b\w\b', ' ', text ) # удаление отдельно стоящих букв

    text = re.sub( r'\b\d+\b', ' digit ', text ) # замена цифр 

    return  text

def train_test_split( data, validation_split = 0.2):
    dict_data = data.to_dict('list')
    sz = len(dict_data['TOPIC'])
    indices = np.arange(sz)
    np.random.shuffle(indices)

    X = [ dict_data['DESCRIPTION'][i] for i in indices ]
    Y = [ dict_data['TOPIC'][i] for i in indices ]
    nb_validation_samples = int( validation_split * sz )

    return { 
        'train': { 'x': X[:-nb_validation_samples], 'y': Y[:-nb_validation_samples]  },
        'test': { 'x': X[-nb_validation_samples:], 'y': Y[-nb_validation_samples:]  }
    }

In [3]:
# Загружаем данные
#raw_data = pd.read_csv('complaints.csv', header = 0, sep = ';')
raw_data = pd.read_excel('complaints.xlsx', sheet_name='complaints', index_col=None)
#raw_data.info()
raw_data.head()

,ISSUENUM,TOPIC,UNDER_TOPIC,REPORTER,ASSIGNEE,CREATOR,STATUS_ISSUE,CREATED,UPDATED,RESOLUTIONDATE,...,FIO_CLIENT,IIN,TEXT_CONTRACT_NUMBER,PHONE_NUMBER,INF_CLIENT_RESPONDING,INCIDENT_REGION,INCIDENT_RESOLVED,FIO_GUILT_WORK,AUTHOR_REGION,AUTHOR_SUBDIVISIONS
0,105578,ЖАЛОБА: на корреспонденцию банка,Cмс по предложениям Xsell,aoryntaeva,akalabaeva,aoryntaeva,Закрыта,2017-12-12 06:47:45,2017-12-12 12:02:50,2017-12-12 12:02:50,...,АҚЖИГИТОВ БЕЙБІТ КЕНЖЕБАЙҰЛЫ,910429301833,0,7776121291,NaN,0,Не требует создания проблемы,NaN,Алматы,Call-center
1,105582,ЖАЛОБА: на корреспонденцию банка,Cмс по предложениям Xsell,atusipova,akalabaeva,atusipova,Закрыта,2017-12-12 08:34:06,2017-12-12 15:41:16,2017-12-12 12:05:45,...,Тверитинова Анна Егоровна,761123400947,3708168706,87078452136 \t,NaN,0,Не требует создания проблемы,NaN,NaN,Call-center
2,105604,ЖАЛОБА: на обслуживание,NaN,azhalgaeva,zissayev,azhalgaeva,Закрыта,2017-12-12 09:27:28,2017-12-21 09:10:29,2017-12-14 16:05:54,...,Дарипова Айгуль Сабыргалиевна,810420403003,0,7292250739,Не дозвон,"Мангистауская область,Бейнеу, ул. Ерконай, д. 1",Не требует создания проблемы,NaN,Актау,Call-center
3,105614,ЖАЛОБА: на обслуживание,ЖАЛОБА: на обслуживание Call-center,aamankeldi,zissayev,aamankeldi,Закрыта,2017-12-12 09:48:57,2018-01-04 08:52:08,2017-12-14 16:06:10,...,Белова Викторовна Татьяна,860714450805,3712793921,7777813881,Не дозвон,Алматы,Не требует создания проблемы,dtungyshbaeva,Алматы,Call-center
4,105632,ЖАЛОБА: на корреспонденцию банка,Cмс по предложениям Xsell,aabdykadyrova,zissayev,aabdykadyrova,Закрыта,2017-12-12 10:34:31,2017-12-14 09:57:35,2017-12-14 09:57:35,...,Онгарбаева Нурсакип Окановна,580209400606,3647319049,8 775 229 15 13,Не дозвон,Алматинская обл,Не требует создания проблемы,NaN,Алматы,Call-center


In [4]:
print("Размерность до препроцессинга: ", raw_data.shape)

#Preprocessing
valuable_columns = ["TOPIC", "DESCRIPTION"]
raw_data = raw_data[valuable_columns]
raw_data = raw_data.dropna(subset=['TOPIC']) # Удаляем строки с пустыми значениями в столбце 'TOPIC'

print("Размерность после препроцессинга", raw_data.shape)

# Отберем только строки, содержащие жалобы, в которых столбец 'TOPIC' содержит различные сочетания слова "жалоба"
raw_data = raw_data[(raw_data.TOPIC.str.contains("ЖАЛОБА: ") |
                     raw_data.TOPIC.str.contains("Жалоба на ") |
                     raw_data.TOPIC.str.contains("Жалобы на "))] #, na=False

print("Размерность данных с Жалобами", raw_data.shape)

raw_data["TOPIC"] = [ label_cleaner(t) for t in raw_data["TOPIC"]] # Почистим метки от "мусора"

print("\tКоличество категорий:", len(set(raw_data["TOPIC"])))
print("\tМножество категорий:")
print(set(raw_data["TOPIC"]))

Размерность до препроцессинга:  (22840, 21)
Размерность после препроцессинга (22805, 2)
Размерность данных с Жалобами (19415, 2)
	Количество категорий: 9
	Множество категорий:
{'корреспонденцию банка', 'обслуживание', 'карточные продукты', 'го/филиал/отделения/микроофисы/тт', 'не согласие с условиями договора, %%, задолженностью, штрафами, тарифами и комиссиями', 'услугу "страховку"', 'услугу "хранитель"', 'услугу "защита семьи"', 'обслуживание в терминалах'}


In [5]:
#Разделяем данные на Тренировочную и Тестовую выборки
D = train_test_split(raw_data, 0.3)

print("\tРазмер тестовой выборки:", len(D['test'] ['y']))
print("\tРазмер тренировочной выборки:", len(D['train'] ['y']))
print("\tОбщий размер данных:", len(D['test'] ['y']) + len(D['train'] ['y']))

	Размер тестовой выборки: 5824
	Размер тренировочной выборки: 13591
	Общий размер данных: 19415


In [6]:
print("[i] Обучение классификатора...")

    # text_clf = Pipeline([
    #                ('hashvect', HashingVectorizer() ),
    #                ('tfidf', TfidfTransformer(use_idf=False )),
    #                ('clf', SGDClassifier(loss='hinge')),
    #                ])
    #
    # text_clf = Pipeline([
    #                ('covect', CountVectorizer() ),
    #                ('tfidf', TfidfTransformer(preprocessor=text_cleaner,use_idf=False )),
    #                ('clf', SGDClassifier(loss='hinge')),
    #                ])


text_clf = Pipeline([
                ('tfidf', TfidfVectorizer()),
                ('clf', SGDClassifier(loss='hinge')),
                ])

text_clf.fit(D['train']['x'], D['train']['y'])

print("[i] Обучение завершено!")

[i] Обучение классификатора...


D:\tools\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


[i] Обучение завершено!


In [7]:
#Testing and checking results
print("[i] Тестируем...")

predicted = text_clf.predict( D['train']['x'] )
print("\taccuracy train: ", accuracy_score(  D['train']['y'] , predicted) )
    
predicted = text_clf.predict( D['test']['x'] )
print("\taccuracy test: ", accuracy_score(  D['test']['y'] , predicted) )

[i] Тестируем...
	accuracy train:  0.9156059156794938
	accuracy test:  0.8597184065934066


In [8]:
frameTrain = pd.DataFrame({'cathegory': D['train']['y'], 'content': D['train']['x']})

In [9]:
frameTrain.head()

,cathegory,content
0,обслуживание,Клиент жалуется на обслуживание в колл-центра....
1,"не согласие с условиями договора, %%, задолжен...","ДД!\n условия кредита, на сайте было указанно ..."
2,обслуживание,клиент жалуется что в раннем оформлении не пре...
3,корреспонденцию банка,Клиент Недосеикин Сергей Лионидович просит \nч...
4,обслуживание,Внешние запросы \n\nФИО: Казанга...


In [10]:
print(frameTrain.shape)

(13591, 2)


In [11]:
frameTrain['predicted'] = text_clf.predict( D['train']['x'] ).tolist()

In [12]:
print(frameTrain.shape)

(13591, 3)


In [13]:
frameTrain.head(50)

,cathegory,content,predicted
0,обслуживание,Клиент жалуется на обслуживание в колл-центра....,обслуживание
1,"не согласие с условиями договора, %%, задолжен...","ДД!\n условия кредита, на сайте было указанно ...",обслуживание
2,обслуживание,клиент жалуется что в раннем оформлении не пре...,обслуживание
3,корреспонденцию банка,Клиент Недосеикин Сергей Лионидович просит \nч...,корреспонденцию банка
4,обслуживание,Внешние запросы \n\nФИО: Казанга...,обслуживание
5,обслуживание,\n\n \n\nВнешние запросы\n\n \n\nФИО: Нет дан...,обслуживание
6,обслуживание,ДД! Клиент Кузнецова Ирина Евгеньевна жалуется...,обслуживание
7,обслуживание,"Менеджер неверно предоставил информацию, ""офор...",обслуживание
8,"не согласие с условиями договора, %%, задолжен...",Добрый день!\nКлиент не согласен с суммой кото...,"не согласие с условиями договора, %%, задолжен..."
9,обслуживание,клиент жалуеться что оплачивал все вовремя и ...,обслуживание


In [14]:
frameTrain.to_excel('complaints_train.xlsx', sheet_name='complaints_train')

In [15]:
frameTest = pd.DataFrame({'cathegory': D['test']['y'], 'content': D['test']['x']})

In [16]:
print(frameTest.shape)

(5824, 2)


In [17]:
frameTest['predicted'] = text_clf.predict( D['test']['x'] ).tolist()

In [18]:
print(frameTest.shape)

(5824, 3)


In [19]:
frameTest.head(50)

,cathegory,content,predicted
0,корреспонденцию банка,"Руководитель мед центра ""Саумет"" Тохсеитова С...",корреспонденцию банка
1,корреспонденцию банка,"ДД! Клиент жалуется на то, что ему приходят см...",корреспонденцию банка
2,корреспонденцию банка,\n\nЗвонят с банка третьему лицу. Просят Макее...,корреспонденцию банка
3,обслуживание,Токмухамбетова Алмагуль Сериковна подала жало...,обслуживание
4,обслуживание,Кайырлы кун ! Клиент шагым калдырды. Менеджер ...,обслуживание
5,обслуживание,Клиент жалуется на менеджера за предоставленну...,обслуживание
6,обслуживание,Клиент посит уточнить почему. Менеджер просит ...,обслуживание
7,обслуживание,"Добрый день, Клиент жалуется на то что при офо...",обслуживание
8,обслуживание,"Дд! У клиента был карточный продукт, однако по...",обслуживание
9,обслуживание,Клиент жалуется на малое количество сотруднико...,обслуживание


In [20]:
frameTest.to_excel('complaints_test.xlsx', sheet_name='complaints_test')